In [1]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()
print("created spark session.")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/21 00:14:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
created spark session.
